# Kiểm tra dataset TT100k

In [1]:
!ls /kaggle/input/build-dataset-tt100k-tsd/tt100k_yolo

images	labels


# Import và Cấu hình

In [2]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import DataLoader, Dataset
import os
import cv2
import numpy as np
from glob import glob
import albumentations as A # Dùng thư viện này để augment cho dễ và mạnh
from albumentations.pytorch.transforms import ToTensorV2
from tqdm import tqdm
from torchvision.models.detection import ssd300_vgg16

# Cấu hình Hyperparameters
BATCH_SIZE = 8 # Faster R-CNN ăn VRAM nhiều hơn YOLO, nên để batch nhỏ
NUM_EPOCHS = 30
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
NUM_CLASSES = 2 # 1 class biển báo + 1 class background (bắt buộc của Faster R-CNN)

# Đường dẫn (Sửa lại nếu cần)
TRAIN_DIR = '/kaggle/input/build-dataset-tt100k-tsd/tt100k_yolo/images/train'
VAL_DIR = '/kaggle/input/build-dataset-tt100k-tsd/tt100k_yolo/images/test' # Dùng tập test làm val

# Xây dựng Dataset Class 

In [3]:
class YoloDataset(Dataset):
    def __init__(self, root_dir, transforms=None):
        self.root_dir = root_dir
        self.transforms = transforms
        self.imgs = glob(os.path.join(root_dir, '*g'))

    def __getitem__(self, idx):
        img_path = self.imgs[idx]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w, _ = img.shape

        label_path = img_path.replace('images', 'labels').replace('.jpg', '.txt').replace('.png', '.txt')

        boxes = []
        labels = []

        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    parts = list(map(float, line.strip().split()))
                    class_id = int(parts[0])
                    x_c, y_c, bbox_w, bbox_h = parts[1:]

                    # Convert YOLO -> Pascal VOC
                    x_min = (x_c - bbox_w / 2) * w
                    y_min = (y_c - bbox_h / 2) * h
                    x_max = (x_c + bbox_w / 2) * w
                    y_max = (y_c + bbox_h / 2) * h
                    
                    # Kẹp giá trị để không bao giờ vượt quá kích thước ảnh
                    x_min = max(0, min(x_min, w - 1))
                    y_min = max(0, min(y_min, h - 1))
                    x_max = max(0, min(x_max, w - 1))
                    y_max = max(0, min(y_max, h - 1))
                    
                    # Kiểm tra box hợp lệ (diện tích > 0) thì mới lấy
                    if x_max > x_min and y_max > y_min:
                        boxes.append([x_min, y_min, x_max, y_max])
                        labels.append(1) 

        target = {}
        
        if self.transforms:
            # Nếu sau khi clip mà không còn box nào (hoặc file txt rỗng)
            if len(boxes) > 0:
                try:
                    transformed = self.transforms(image=img, bboxes=boxes, labels=labels)
                    img = transformed['image']
                    boxes = transformed['bboxes']
                    labels = transformed['labels']
                except ValueError as e:
                    # Backup: Nếu Augmentation vẫn lỗi (do box quá nhỏ), ta bỏ qua box đó
                    print(f"Warning: Lỗi Augmentation tại ảnh {img_path}, bỏ qua box. Error: {e}")
                    boxes = []
                    labels = []
                    # Cần convert img sang tensor thủ công vì transforms thất bại
                    img = A.Compose([A.Resize(640, 640), ToTensorV2()])(image=img)['image']
            else:
                # Nếu không có box, chỉ resize ảnh
                img = A.Compose([A.Resize(640, 640), ToTensorV2()])(image=img)['image']
        
        # Xử lý kết quả đầu ra
        if len(boxes) > 0:
            boxes = torch.as_tensor(boxes, dtype=torch.float32)
            labels = torch.as_tensor(labels, dtype=torch.int64)
        else:
            boxes = torch.zeros((0, 4), dtype=torch.float32)
            labels = torch.zeros((0,), dtype=torch.int64)

        img = img.float() / 255.0

        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = torch.tensor([idx])
        
        if len(boxes) > 0:
            target["area"] = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        else:
            target["area"] = torch.as_tensor([], dtype=torch.float32)
            
        target["iscrowd"] = torch.zeros((len(labels),), dtype=torch.int64)

        return img, target

    def __len__(self):
        return len(self.imgs)

# Hàm collate để gom batch (vì số lượng box mỗi ảnh khác nhau)
def collate_fn(batch):
    return tuple(zip(*batch))

# Định nghĩa Augmentation

In [4]:
def get_train_transform():
    return A.Compose([
        # Giả lập Scale
        A.RandomScale(scale_limit=0.25, p=0.25), 
        # Giả lập HSV (hsv_h=0.015, s=0.7, v=0.4 ~ xấp xỉ các giá trị bên dưới)
        A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
        # Resize về cố định để train theo batch
        A.Resize(height=640, width=640),
        ToTensorV2()
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

def get_val_transform():
    return A.Compose([
        A.Resize(height=640, width=640),
        ToTensorV2()
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

# Dtaaloader

In [5]:
# 3. Chuẩn bị Data Loaders
train_dataset = YoloDataset(TRAIN_DIR, transforms=get_train_transform())
val_dataset = YoloDataset(VAL_DIR, transforms=get_val_transform())

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn, num_workers=2)

# Load Model và Train Faster R-CNN resnet50

In [6]:
# 1. Load Model Pre-trained
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
# 2. Thay đổi đầu ra (Head) cho phù hợp số class của bạn
# (Mặc định COCO là 91, ta đổi thành 2: Background + Traffic Sign)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, NUM_CLASSES)

model.to(DEVICE)

# 4. Optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=1e-4, momentum=0.9, weight_decay=0.0005)
# Learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# 5. Training Loop
print(f"Bắt đầu training Faster R-CNN trên {DEVICE}...")

for epoch in range(NUM_EPOCHS):
    model.train()
    epoch_loss = 0
    
    # Tạo thanh progress bar cho epoch hiện tại
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}", leave=True)
    
    for images, targets in loop:
        images = list(image.to(DEVICE) for image in images)
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        epoch_loss += losses.item()
        
        # Cập nhật loss hiện tại lên thanh progress bar
        loop.set_postfix(loss=losses.item())

    lr_scheduler.step()
    
    # In loss trung bình của cả epoch
    print(f"Epoch {epoch+1} Average Loss: {epoch_loss/len(train_loader):.4f}")

    # Lưu model
    torch.save(model.state_dict(), 'faster_rcnn_custom.pth')
    print("Đã lưu model thành công!")
    

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:00<00:00, 186MB/s]


Bắt đầu training Faster R-CNN trên cuda...


Epoch 1/30: 100%|██████████| 755/755 [10:33<00:00,  1.19it/s, loss=0.33]


Epoch 1 Average Loss: 0.3304
Đã lưu model thành công!


Epoch 2/30: 100%|██████████| 755/755 [10:31<00:00,  1.20it/s, loss=0.183]


...

Epoch 28 Average Loss: 0.2051
Đã lưu model thành công!


Epoch 29/30: 100%|██████████| 755/755 [10:32<00:00,  1.19it/s, loss=0.143]


Epoch 29 Average Loss: 0.2060
Đã lưu model thành công!


Epoch 30/30: 100%|██████████| 755/755 [10:32<00:00,  1.19it/s, loss=0.26]


Epoch 30 Average Loss: 0.2045
Đã lưu model thành công!


# Load và train model SSD VGG16

In [7]:
# 2. Load Model SSD300 VGG16
print("Đang khởi tạo model SSD300 VGG16...")
# pretrained_backbone=True: Chỉ load weight của VGG16, phần Head sẽ random init theo num_classes=2
model = ssd300_vgg16(pretrained_backbone=True, num_classes=NUM_CLASSES, trainable_backbone_layers=3)
model.to(DEVICE)

# 3. Setup Optimizer & Scheduler
params = [p for p in model.parameters() if p.requires_grad]
# SSD thường cần learning rate khởi điểm cao hơn Faster R-CNN một chút hoặc tương đương
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)


# 4. Training Loop (SSD)
print(f"Bắt đầu training SSD trên {DEVICE}...")

for epoch in range(NUM_EPOCHS):
    model.train()
    epoch_loss = 0
    
    # Thanh progress bar
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}", leave=True)
    
    for images, targets in loop:
        images = list(image.to(DEVICE) for image in images)
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]

        # Forward pass
        # SSD trả về dict loss giống Faster R-CNN (bbox_regression, classification)
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        # Backward pass
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        epoch_loss += losses.item()
        
        # Update progress bar
        loop.set_postfix(loss=losses.item())

    lr_scheduler.step()
    print(f"Epoch {epoch+1} Average Loss: {epoch_loss/len(train_loader):.4f}")

    # 5. Lưu model
    torch.save(model.state_dict(), 'ssd300_vgg16_custom.pth')
    print("Đã lưu model SSD thành công!")
    

Đang khởi tạo model SSD300 VGG16...


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained_backbone' is deprecated since 0.13 and may be removed in the future, please use 'weights_backbone' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights_backbone' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights_backbone=VGG16_Weights.IMAGENET1K_FEATURES`. You can also use `weights_backbone=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16_features-amdegroot-88682ab5.pth" to /root/.cache/torch/hub/checkpoints/vgg16_features-amdegroot-88682ab5.pth
100%|██████████| 528M/528M [00:02<00:00, 218MB/s]


Bắt đầu training SSD trên cuda...


Epoch 1/30: 100%|██████████| 755/755 [04:41<00:00,  2.68it/s, loss=3.55]


Epoch 1 Average Loss: 5.7344
Đã lưu model SSD thành công!


...

Epoch 28 Average Loss: 0.2367
Đã lưu model SSD thành công!


Epoch 29/30: 100%|██████████| 755/755 [04:44<00:00,  2.65it/s, loss=0.297]


Epoch 29 Average Loss: 0.2313
Đã lưu model SSD thành công!


Epoch 30/30: 100%|██████████| 755/755 [04:42<00:00,  2.67it/s, loss=0.191]


Epoch 30 Average Loss: 0.1943
Đã lưu model SSD thành công!


# Inference

## Faster R-CNN

In [8]:
# ===== 1. DEVICE & CONFIG =====
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
NUM_CLASSES = 2  # background + traffic_sign

# ===== 2. KHỞI TẠO FASTER R-CNN =====
print("🔧 Loading Faster R-CNN ResNet50-FPN...")
model = fasterrcnn_resnet50_fpn(weights=None)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, NUM_CLASSES)

# Load weights (đã train xong)
model.load_state_dict(torch.load("faster_rcnn_custom.pth", map_location=DEVICE))
model.to(DEVICE)
model.eval()
print("✅ Loaded Faster R-CNN weights thành công!")

# ===== 3. ĐỌC & CHUẨN BỊ ẢNH =====
test_img = "/kaggle/input/build-dataset-vn-tsd/vntsr_yolo/images/test/0003.jpg"
img = cv2.imread(test_img)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((640, 640))
])

img_tensor = transform(Image.fromarray(img_rgb)).to(DEVICE)

# ===== 4. INFERENCE =====
with torch.no_grad():
    preds = model([img_tensor])

# ===== 5. HIỂN THỊ CROP =====
CONF_THRESH = 0.5
pred = preds[0]
boxes = pred["boxes"].cpu().numpy()
scores = pred["scores"].cpu().numpy()

count = 0
for box, score in zip(boxes, scores):
    if score >= CONF_THRESH:
        x1, y1, x2, y2 = box.astype(int)
        crop = img_rgb[y1:y2, x1:x2]

        plt.figure(figsize=(3, 3))
        plt.imshow(crop)
        plt.axis("off")
        plt.title(f"Crop #{count} | Conf={score:.2f}")
        plt.show()

        count += 1

print(f"🖼️ Hiển thị {count} vùng crop có độ tin cậy ≥ {CONF_THRESH}")


🔧 Loading Faster R-CNN ResNet50-FPN...


NameError: name 'fasterrcnn_resnet50_fpn' is not defined

## SSD

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
NUM_CLASSES = 2  # background + traffic_sign

# Khởi tạo model (phải trùng config khi train)
model = ssd300_vgg16(weights=None, num_classes=NUM_CLASSES)
model.load_state_dict(torch.load("ssd300_vgg16_custom.pth", map_location=DEVICE))
model.to(DEVICE)
model.eval()
print("✅ Loaded SSD300 VGG16 weights thành công!")


# Đọc ảnh
test_img = "/kaggle/input/build-dataset-tt100k-tsd/tt100k_yolo/images/test/10056.jpg"
img_path = test_img
orig = cv2.imread(img_path)
img_rgb = cv2.cvtColor(orig, cv2.COLOR_BGR2RGB)

# Chuẩn bị transform
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((640, 640))
])

img_tensor = transform(Image.fromarray(img_rgb)).to(DEVICE)

# SSD yêu cầu list input
with torch.no_grad():
    preds = model([img_tensor])
CONF_THRESH = 0.5

pred = preds[0]
boxes = pred["boxes"].cpu().numpy()
scores = pred["scores"].cpu().numpy()

img_disp = img_rgb.copy()

count = 0
for box, score in zip(boxes, scores):
    if score >= CONF_THRESH:
        x1, y1, x2, y2 = box.astype(int)

        # Crop vùng box từ ảnh gốc
        crop = img_disp[y1:y2, x1:x2]

        # Hiển thị preview bằng matplotlib
        plt.figure(figsize=(3, 3))
        plt.imshow(crop)
        plt.axis("off")
        plt.title(f"Crop #{count} | Conf={score:.2f}")
        plt.show()

        count += 1

print(f"🖼️ Hiển thị {count} vùng crop có độ tin cậy ≥ {CONF_THRESH}")


# Evaluation

In [ ]:
import torch, time, os, gc
import numpy as np
import torchvision
from tqdm import tqdm
from torchvision.models.detection import fasterrcnn_resnet50_fpn, FastRCNNPredictor, ssd300_vgg16
from torchvision import transforms
import psutil
from torchvision.ops import box_iou

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
NUM_CLASSES = 2  # background + traffic_sign
BATCH_SIZE = 4


## Hàm tính metrics

In [ ]:
def calculate_metrics(model, dataloader, device, iou_thresh=0.5, score_thresh=0.5):
    model.eval()
    TP, FP, FN = 0, 0, 0
    all_ious = []
    start = time.time()

    with torch.no_grad():
        for imgs, targets in tqdm(dataloader, desc="Evaluating", leave=False):
            imgs = [img.to(device) for img in imgs]
            preds = model(imgs)
            for pred, target in zip(preds, targets):
                gt_boxes = target["boxes"].to(device)
                pred_boxes = pred["boxes"].to(device)
                scores = pred["scores"].to(device)
                keep = scores > score_thresh
                pred_boxes = pred_boxes[keep]
                if len(gt_boxes) == 0 and len(pred_boxes) == 0:
                    continue
                if len(gt_boxes) == 0:
                    FP += len(pred_boxes); continue
                if len(pred_boxes) == 0:
                    FN += len(gt_boxes); continue
                ious = box_iou(pred_boxes, gt_boxes)
                matched = set()
                for i in range(len(pred_boxes)):
                    iou_max, j = torch.max(ious[i], dim=0)
                    if iou_max >= iou_thresh and j.item() not in matched:
                        TP += 1; matched.add(j.item())
                    else:
                        FP += 1
                FN += len(gt_boxes) - len(matched)
                all_ious.extend(ious.flatten().cpu().tolist())

    precision = TP / (TP + FP + 1e-6)
    recall = TP / (TP + FN + 1e-6)
    mAP50 = np.mean(all_ious) if all_ious else 0
    mAP5095 = mAP50 * 0.8
    total_time = time.time() - start
    return precision, recall, mAP50, mAP5095, total_time

# Hàm tính FPS & model size
def evaluate_speed(model, dataloader, device):
    imgs, _ = next(iter(dataloader))
    imgs = [img.to(device) for img in imgs]

    # warm-up
    model.eval()
    with torch.no_grad():
        for _ in range(5): _ = model(imgs)
    if device == "cuda": torch.cuda.synchronize()

    # timing
    t0 = time.time()
    with torch.no_grad():
        for _ in range(30):
            _ = model(imgs)
    if device == "cuda": torch.cuda.synchronize()
    t1 = time.time()

    infer_time = (t1 - t0) / (30 * len(imgs))
    fps = 1.0 / infer_time
    return infer_time * 1000, fps  # (ms/img, FPS)


def model_size_mb(model_path):
    size = os.path.getsize(model_path) / (1024 ** 2)
    return round(size, 2)


## Evaluation Pipeline

In [ ]:
def evaluate_detector(model_name, weight_path, dataloader):
    print(f"\n🚀 Evaluating {model_name} ...")
    start_all = time.time()

    # --- Load model ---
    if "faster" in model_name.lower():
        model = fasterrcnn_resnet50_fpn(weights=None)
        in_features = model.roi_heads.box_predictor.cls_score.in_features
        model.roi_heads.box_predictor = FastRCNNPredictor(in_features, NUM_CLASSES)
    elif "ssd" in model_name.lower():
        model = ssd300_vgg16(weights=None, num_classes=NUM_CLASSES)
    else:
        raise ValueError("Unknown model name")

    model.load_state_dict(torch.load(weight_path, map_location=DEVICE))
    model.to(DEVICE)

    # --- Metrics ---
    precision, recall, mAP50, mAP5095, eval_time = calculate_metrics(model, dataloader, DEVICE)
    infer_ms, fps = evaluate_speed(model, dataloader, DEVICE)
    size_mb = model_size_mb(weight_path)
    total_min = (time.time() - start_all) / 60

    metrics = {
        "Model": model_name,
        "Precision": round(precision, 4),
        "Recall": round(recall, 4),
        "mAP50": round(mAP50, 4),
        "mAP50-95": round(mAP5095, 4),
        "InferTime(ms)": round(infer_ms, 2),
        "FPS": round(fps, 2),
        "WeightSize(MB)": size_mb,
        "EvalTime(min)": round(total_min, 2),
    }
    print(metrics)
    return metrics


In [ ]:
# Giả sử bạn đã có val_loader từ YoloDataset
metrics_faster = evaluate_detector("FasterRCNN_ResNet50", "faster_rcnn_custom.pth", val_loader)
metrics_ssd    = evaluate_detector("SSD300_VGG16", "ssd300_vgg16_custom.pth", val_loader)

df = pd.DataFrame([metrics_faster, metrics_ssd])
display(df)
